# ⚡ Lecture 11: Efficient Transformers - Complete Demo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gaurav-redhat/efficientml_course/blob/main/11_efficient_transformers/demo.ipynb)

## What You'll Learn
- Attention complexity analysis (O(N²) problem)
- Linear attention mechanisms
- FlashAttention and memory efficiency
- Sparse attention patterns

In [ ]:
!pip install torch matplotlib numpy -q
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import time

torch.manual_seed(42)
print('Ready for Efficient Transformers!')

## Part 1: The O(N²) Problem

Standard attention: $\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d}}\right)V$

The $QK^T$ matrix is $N \times N$ → O(N²) memory and compute!

In [ ]:
def standard_attention(Q, K, V):
    """
    Standard scaled dot-product attention.
    Memory: O(N²) for attention matrix
    Compute: O(N² × d)
    """
    d_k = Q.shape[-1]
    scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(d_k)  # N × N matrix!
    attn_weights = F.softmax(scores, dim=-1)
    return torch.matmul(attn_weights, V), attn_weights

# Analyze scaling
seq_lengths = [128, 256, 512, 1024, 2048, 4096]
d_model = 64

print('📊 ATTENTION COMPLEXITY SCALING')
print('=' * 60)
print(f'{"Seq Length":<12} {"Memory (MB)":<15} {"Time (ms)":<15}')
print('-' * 60)

memory_usage = []
time_usage = []

for seq_len in seq_lengths:
    Q = torch.randn(1, seq_len, d_model)
    K = torch.randn(1, seq_len, d_model)
    V = torch.randn(1, seq_len, d_model)
    
    # Memory for attention matrix (N × N × 4 bytes for float32)
    attn_memory_mb = seq_len * seq_len * 4 / 1024 / 1024
    memory_usage.append(attn_memory_mb)
    
    # Time measurement
    start = time.time()
    for _ in range(10):
        _, _ = standard_attention(Q, K, V)
    elapsed = (time.time() - start) / 10 * 1000
    time_usage.append(elapsed)
    
    print(f'{seq_len:<12} {attn_memory_mb:<15.2f} {elapsed:<15.2f}')

print('\n⚠️ Memory grows QUADRATICALLY with sequence length!')

In [ ]:
# Visualize quadratic scaling
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Memory
axes[0].plot(seq_lengths, memory_usage, 'o-', color='#ef4444', linewidth=2, markersize=10)
axes[0].fill_between(seq_lengths, memory_usage, alpha=0.3, color='#ef4444')
axes[0].set_xlabel('Sequence Length')
axes[0].set_ylabel('Memory (MB)')
axes[0].set_title('📊 Attention Memory: O(N²)')
axes[0].grid(True, alpha=0.3)

# Add quadratic reference
x_ref = np.array(seq_lengths)
y_ref = (x_ref / seq_lengths[0]) ** 2 * memory_usage[0]
axes[0].plot(x_ref, y_ref, '--', color='gray', label='O(N²) reference')
axes[0].legend()

# Time
axes[1].plot(seq_lengths, time_usage, 'o-', color='#3b82f6', linewidth=2, markersize=10)
axes[1].fill_between(seq_lengths, time_usage, alpha=0.3, color='#3b82f6')
axes[1].set_xlabel('Sequence Length')
axes[1].set_ylabel('Time (ms)')
axes[1].set_title('📊 Attention Time: O(N² × d)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Real-world implications
print('\n📊 REAL-WORLD IMPLICATIONS')
print('=' * 50)
contexts = [4096, 32000, 128000, 1000000]
for ctx in contexts:
    mem_gb = ctx * ctx * 4 / 1024**3
    print(f'{ctx:>10} tokens: {mem_gb:>10.2f} GB for attention matrix')

## Part 2: Linear Attention

In [ ]:
def linear_attention(Q, K, V, eps=1e-6):
    """
    Linear attention using kernel trick.
    
    Key insight: softmax(QK^T)V ≈ φ(Q)(φ(K)^T V)
    
    Instead of N×N attention matrix, compute:
    1. φ(K)^T V: d×d matrix (small!)
    2. φ(Q) × result: N×d
    
    Memory: O(N×d) instead of O(N²)
    """
    # Simple kernel: ELU + 1 (ensures positivity)
    Q_prime = F.elu(Q) + 1
    K_prime = F.elu(K) + 1
    
    # Compute K^T V first: (d × N) @ (N × d) = d × d
    KV = torch.matmul(K_prime.transpose(-2, -1), V)  # d × d matrix!
    
    # Normalization factor
    K_sum = K_prime.sum(dim=-2, keepdim=True)  # 1 × d
    
    # Compute output: (N × d) @ (d × d) = N × d
    numerator = torch.matmul(Q_prime, KV)
    denominator = torch.matmul(Q_prime, K_sum.transpose(-2, -1)) + eps
    
    return numerator / denominator

# Compare standard vs linear attention
print('📊 STANDARD vs LINEAR ATTENTION')
print('=' * 60)
print(f'{"Seq Length":<12} {"Standard (ms)":<15} {"Linear (ms)":<15} {"Speedup":<10}')
print('-' * 60)

standard_times = []
linear_times = []

for seq_len in [256, 512, 1024, 2048, 4096]:
    Q = torch.randn(1, seq_len, d_model)
    K = torch.randn(1, seq_len, d_model)
    V = torch.randn(1, seq_len, d_model)
    
    # Standard
    start = time.time()
    for _ in range(10):
        _ = standard_attention(Q, K, V)
    std_time = (time.time() - start) / 10 * 1000
    standard_times.append(std_time)
    
    # Linear
    start = time.time()
    for _ in range(10):
        _ = linear_attention(Q, K, V)
    lin_time = (time.time() - start) / 10 * 1000
    linear_times.append(lin_time)
    
    speedup = std_time / lin_time
    print(f'{seq_len:<12} {std_time:<15.2f} {lin_time:<15.2f} {speedup:<10.1f}x')

In [ ]:
# Visualize comparison
seq_lens = [256, 512, 1024, 2048, 4096]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Time comparison
x = np.arange(len(seq_lens))
width = 0.35

axes[0].bar(x - width/2, standard_times, width, label='Standard O(N²)', color='#ef4444')
axes[0].bar(x + width/2, linear_times, width, label='Linear O(N)', color='#22c55e')
axes[0].set_xlabel('Sequence Length')
axes[0].set_ylabel('Time (ms)')
axes[0].set_title('📊 Attention Time Comparison')
axes[0].set_xticks(x)
axes[0].set_xticklabels(seq_lens)
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# Scaling comparison (log scale)
axes[1].semilogy(seq_lens, standard_times, 'o-', label='Standard O(N²)', color='#ef4444', linewidth=2)
axes[1].semilogy(seq_lens, linear_times, 's-', label='Linear O(N)', color='#22c55e', linewidth=2)
axes[1].set_xlabel('Sequence Length')
axes[1].set_ylabel('Time (ms, log scale)')
axes[1].set_title('📊 Scaling Behavior')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Part 3: Sparse Attention Patterns

In [ ]:
def create_attention_patterns(seq_len):
    """
    Create different sparse attention patterns.
    """
    patterns = {}
    
    # Full attention
    patterns['Full'] = torch.ones(seq_len, seq_len)
    
    # Local/sliding window (each token attends to neighbors)
    window_size = seq_len // 8
    local = torch.zeros(seq_len, seq_len)
    for i in range(seq_len):
        start = max(0, i - window_size)
        end = min(seq_len, i + window_size + 1)
        local[i, start:end] = 1
    patterns['Local (Window)'] = local
    
    # Strided/dilated
    stride = 4
    strided = torch.zeros(seq_len, seq_len)
    for i in range(seq_len):
        strided[i, ::stride] = 1  # Every stride-th position
        strided[i, max(0, i-2):min(seq_len, i+3)] = 1  # Local context
    patterns['Strided'] = strided
    
    # Global + Local (Longformer style)
    global_local = local.clone()
    num_global = 4
    global_local[:num_global, :] = 1  # First tokens are global
    global_local[:, :num_global] = 1
    patterns['Global+Local'] = global_local
    
    # Block sparse (BigBird style)
    block_size = seq_len // 8
    block_sparse = torch.zeros(seq_len, seq_len)
    for i in range(0, seq_len, block_size):
        block_sparse[i:i+block_size, i:i+block_size] = 1
    # Add some random blocks
    for _ in range(seq_len // block_size):
        ri = np.random.randint(0, seq_len // block_size) * block_size
        rj = np.random.randint(0, seq_len // block_size) * block_size
        block_sparse[ri:ri+block_size, rj:rj+block_size] = 1
    patterns['Block Sparse'] = block_sparse
    
    return patterns

# Visualize patterns
seq_len = 64
patterns = create_attention_patterns(seq_len)

fig, axes = plt.subplots(1, 5, figsize=(20, 4))

for ax, (name, pattern) in zip(axes, patterns.items()):
    sparsity = 100 * (1 - pattern.sum() / pattern.numel())
    ax.imshow(pattern, cmap='Blues', aspect='auto')
    ax.set_title(f'{name}\n{sparsity:.1f}% sparse')
    ax.set_xlabel('Key Position')
    ax.set_ylabel('Query Position')

plt.suptitle('📊 Sparse Attention Patterns', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
def sparse_attention(Q, K, V, mask):
    """
    Attention with sparse mask.
    
    In practice, this is implemented more efficiently
    using block-sparse operations or FlashAttention.
    """
    d_k = Q.shape[-1]
    scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(d_k)
    
    # Apply sparse mask
    scores = scores.masked_fill(mask == 0, float('-inf'))
    
    attn_weights = F.softmax(scores, dim=-1)
    attn_weights = attn_weights.masked_fill(mask == 0, 0)  # Clean up NaNs
    
    return torch.matmul(attn_weights, V)

# Compare accuracy vs efficiency trade-off
print('📊 SPARSE ATTENTION EFFICIENCY')
print('=' * 60)

seq_len = 256
Q = torch.randn(1, seq_len, d_model)
K = torch.randn(1, seq_len, d_model)
V = torch.randn(1, seq_len, d_model)

# Full attention output (ground truth)
full_out, _ = standard_attention(Q, K, V)

patterns = create_attention_patterns(seq_len)

print(f'{"Pattern":<20} {"Sparsity (%)":<15} {"MSE vs Full":<15} {"Speedup":<10}')
print('-' * 60)

for name, mask in patterns.items():
    sparsity = 100 * (1 - mask.sum() / mask.numel())
    
    # Compute sparse attention
    sparse_out = sparse_attention(Q, K, V, mask.unsqueeze(0))
    
    # Measure difference from full attention
    mse = torch.mean((full_out - sparse_out) ** 2).item()
    
    # Theoretical speedup (proportional to non-zero entries)
    speedup = mask.numel() / mask.sum().item()
    
    print(f'{name:<20} {sparsity:<15.1f} {mse:<15.6f} {speedup:<10.1f}x')

## Part 4: FlashAttention Concept

In [ ]:
def flash_attention_demo(Q, K, V, block_size=64):
    """
    Simplified FlashAttention demonstration.
    
    Key ideas:
    1. Compute attention in blocks to fit in fast SRAM
    2. Never materialize full N×N attention matrix
    3. Use online softmax (running max and sum)
    
    This is a simplified version - real FlashAttention uses CUDA kernels.
    """
    batch, seq_len, d = Q.shape
    
    # Output and running statistics
    O = torch.zeros_like(V)
    L = torch.zeros(batch, seq_len, 1)  # Log-sum-exp
    M = torch.full((batch, seq_len, 1), float('-inf'))  # Running max
    
    # Process in blocks
    num_blocks = (seq_len + block_size - 1) // block_size
    
    for j in range(num_blocks):
        # Key-Value block
        kv_start = j * block_size
        kv_end = min((j + 1) * block_size, seq_len)
        K_block = K[:, kv_start:kv_end, :]
        V_block = V[:, kv_start:kv_end, :]
        
        for i in range(num_blocks):
            # Query block
            q_start = i * block_size
            q_end = min((i + 1) * block_size, seq_len)
            Q_block = Q[:, q_start:q_end, :]
            
            # Compute block attention scores
            S_block = torch.matmul(Q_block, K_block.transpose(-2, -1)) / np.sqrt(d)
            
            # Online softmax update
            M_block = S_block.max(dim=-1, keepdim=True).values
            P_block = torch.exp(S_block - M_block)
            L_block = P_block.sum(dim=-1, keepdim=True)
            
            # Update running statistics
            M_new = torch.maximum(M[:, q_start:q_end, :], M_block)
            
            exp_diff_old = torch.exp(M[:, q_start:q_end, :] - M_new)
            exp_diff_new = torch.exp(M_block - M_new)
            
            L_new = exp_diff_old * L[:, q_start:q_end, :] + exp_diff_new * L_block
            
            # Update output
            O[:, q_start:q_end, :] = (
                exp_diff_old * L[:, q_start:q_end, :] * O[:, q_start:q_end, :] +
                exp_diff_new * torch.matmul(P_block, V_block)
            ) / (L_new + 1e-8)
            
            # Update statistics
            M[:, q_start:q_end, :] = M_new
            L[:, q_start:q_end, :] = L_new
    
    return O

# Compare outputs
print('📊 FLASHATTENTION DEMONSTRATION')
print('=' * 50)

Q = torch.randn(1, 256, 64)
K = torch.randn(1, 256, 64)
V = torch.randn(1, 256, 64)

# Standard attention
standard_out, _ = standard_attention(Q, K, V)

# Flash attention (simplified)
flash_out = flash_attention_demo(Q, K, V, block_size=32)

# Compare
mse = torch.mean((standard_out - flash_out) ** 2).item()
print(f'MSE between standard and Flash: {mse:.8f}')
print(f'Outputs match: {mse < 1e-5}')

print('\n💡 FlashAttention benefits:')
print('   - 2-4x faster than standard attention')
print('   - Memory: O(N) instead of O(N²)')
print('   - Enables much longer contexts')

## Part 5: Efficient Attention Summary

In [ ]:
# Summary comparison
methods = {
    'Standard': {'complexity': 'O(N²)', 'memory': 'O(N²)', 'exact': 'Yes', 'contexts': '2-8K'},
    'Linear (Performer)': {'complexity': 'O(N)', 'memory': 'O(N)', 'exact': 'Approx', 'contexts': '64K+'},
    'Sparse (Longformer)': {'complexity': 'O(N√N)', 'memory': 'O(N√N)', 'exact': 'Approx', 'contexts': '16K'},
    'FlashAttention': {'complexity': 'O(N²)', 'memory': 'O(N)', 'exact': 'Yes', 'contexts': '32K+'},
    'Ring Attention': {'complexity': 'O(N²/P)', 'memory': 'O(N/P)', 'exact': 'Yes', 'contexts': '1M+'},
}

print('📊 EFFICIENT ATTENTION METHODS')
print('=' * 80)
print(f'{"Method":<20} {"Time":<12} {"Memory":<12} {"Exact?":<10} {"Context":<12}')
print('-' * 80)
for name, info in methods.items():
    print(f'{name:<20} {info["complexity"]:<12} {info["memory"]:<12} {info["exact"]:<10} {info["contexts"]:<12}')

# Visualize
fig, ax = plt.subplots(figsize=(12, 6))

# Simulated scaling
seq_lens = np.array([1024, 2048, 4096, 8192, 16384, 32768])

scaling = {
    'Standard': seq_lens ** 2 / seq_lens[0] ** 2,
    'Linear': seq_lens / seq_lens[0],
    'Sparse': seq_lens * np.sqrt(seq_lens) / (seq_lens[0] * np.sqrt(seq_lens[0])),
    'FlashAttention': seq_lens ** 2 / seq_lens[0] ** 2 / 3,  # Same complexity but faster
}

colors = ['#ef4444', '#22c55e', '#3b82f6', '#f59e0b']

for (name, scale), color in zip(scaling.items(), colors):
    ax.semilogy(seq_lens, scale, 'o-', label=name, linewidth=2, markersize=8, color=color)

ax.set_xlabel('Sequence Length')
ax.set_ylabel('Relative Time (log scale)')
ax.set_title('📊 Attention Method Scaling')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
print('🎯 KEY TAKEAWAYS')
print('=' * 60)
print('\n1. Standard attention is O(N²) - limits context length')
print('\n2. Linear attention: O(N) via kernel trick, approximate')
print('\n3. Sparse attention: Skip some pairs, structured patterns')
print('\n4. FlashAttention: O(N) memory, exact, uses tiling')
print('\n5. Choose method based on accuracy vs efficiency needs')
print('\n6. Modern LLMs combine multiple techniques')
print('\n' + '=' * 60)
print('\n📚 Next: Efficient Training!')